In [93]:
from resources.filterLang import FilterLanguage as FL
from resources.tokTT import CommentTokenizer as CT 
import fasttext
import numpy as np
from resources.basicIO import InputOutput as IO

In [94]:
def NN(model, line, K):
    return model.get_nearest_neighbors(line, k=K)

In [95]:
# finds similarity score 
def score(model, line,k=10):
    # words contains all the words in the corpus
    lst1 = NN(model, line, k)

    v1 = []

    l1 = [x[1] for x in lst1]

    l10 = [x[0] for x in lst1]

    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)

    return v1


In [96]:
def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


In [97]:
def fun(x,y):
    return cos_sim(x,y)

In [98]:
model_N_2 = fasttext.load_model('models/ft_unsupervised_N_2.bin')

In [99]:
# loading from 200th comments
expansion_text = IO.load_csv_col('datasets/random_sample.csv','comment')
expansion_text = expansion_text[0:800]
expansion_text_labels = IO.load_csv_col('datasets/random_sample.csv','label')
expansion_text_labels = list(map(str,map(int,expansion_text_labels[0:800])))
seed_set_text = IO.load_text('datasets/seed_set.txt')
seed_set_labels = IO.load_text('datasets/seed_set_labels.txt')
seed_set_TK = CT.cleaned('datasets/seed_set.txt')
expansion_TK = [CT.tokenize(x) for x in expansion_text]


In [100]:
score_ex = []
i=0
for line in expansion_TK:
    score_ex.append(score(model_N_2,line,20))
    i+=1
    if(i==200):
        break

In [101]:
A = np.array(score_ex)
B = np.array(score_ex).T
C = np.zeros((200, 200))

In [102]:
m,n = C.shape
for i in range(m):
    for j in range(n):
        C[i][j] = fun(A[i,:], B[:,j])

In [103]:
print(C)

[[1.         0.19565166 0.10761619 ... 0.10575569 0.15847286 0.        ]
 [0.19565166 1.         0.24934456 ... 0.24671294 0.35776143 0.        ]
 [0.10761619 0.24934456 1.         ... 0.57480697 0.37556778 0.        ]
 ...
 [0.10575569 0.24671294 0.57480697 ... 1.         0.41346821 0.        ]
 [0.15847286 0.35776143 0.37556778 ... 0.41346821 1.         0.05346978]
 [0.         0.         0.         ... 0.         0.05346978 1.        ]]
